In [2]:

import csv
import json
import pandas as pd
import os
from datetime import datetime
from pytz import timezone
import matplotlib.pyplot as plt
import datetime
import urllib
import numpy as np
import plotly.graph_objects as go

Loading the data and generating basic stats

In [32]:

df = pd.read_csv("raw_data/raw_politcal_posts_two.csv")

print("Basic Information")
print(df.columns)
print(df.shape)
#print(df.head())

# count the number of post per subreddit
print(df['subreddit'].value_counts())

# all of the subreddits that are included
print(df['subreddit'].unique())


Basic Information
Index(['id', 'subreddit', 'title', 'author', 'url', 'selftext', 'score',
       'num_comments', 'created_utc'],
      dtype='object')
(2413, 9)
Conservative            219
Libertarian             205
Anarcho_Capitalism      203
socialism               201
Anarchism               201
conservatives           126
democrats               122
Republican              119
progressive             115
Liberal                 112
alltheleft              110
GreenParty              110
SocialDemocracy          98
Capitalism               87
Objectivism              60
republicans              53
blackflag                47
Marxism                  39
feminisms                35
labor                    32
democracy                27
LibertarianSocialism     21
IWW                      18
LibertarianLeft          14
Egalitarianism           10
Postleftanarchism         6
neoprogs                  5
americanpirateparty       5
AnarchoPacifism           4
Liberty                   

In [33]:
df.head()

,id,subreddit,title,author,url,selftext,score,num_comments,created_utc
0,10w1dj4,Libertarian,What is the Libertarian solution to climate ch...,ErgonomicSikorsky,https://www.reddit.com/r/Libertarian/comments/...,It’s coming so we got to be prepared for it.,20,315,1.675775e+09
1,aocqp,Libertarian,Prof Anastasios Tsonis: University of Wisconsi...,DougDante,http://www.dailymail.co.uk/sciencetech/article...,NaN,0,2,1.263243e+09
2,v65fv1,Libertarian,Three Supreme Court cases to watch beyond abor...,Elranzer,https://www.brennancenter.org/our-work/researc...,NaN,369,318,1.654527e+09
3,voask2,Libertarian,Supreme Court limits EPA’s power to combat cli...,Careless_Bat2543,https://www.washingtonpost.com/politics/2022/0...,NaN,219,194,1.656602e+09
4,uqcz0p,Libertarian,A green libertarian perspective: is bad to hav...,cr7fan89,https://www.reddit.com/r/Libertarian/comments/...,I consider myself a believer in what is called...,133,468,1.652641e+09


In [34]:
# filter out the rows that selftext count is not bigger than one
df = df[df['selftext'].str.len() > 1]

print(df.shape)


(791, 9)


In [38]:
#Getting a random sample of data
sample_df = df.sample(frac=1/90).reset_index(drop=True)

print(sample_df.head())
print(sample_df.shape)
print(sample_df['subreddit'].value_counts())


sample_df.to_csv("raw_data/sample_political_posts_two.csv", index=False)



       id             subreddit  \
0  mrwcxj  LibertarianSocialism   
1  8ueiqj             Anarchism   
2  y4p3mu       SocialDemocracy   
3  ub927g            Capitalism   
4  kbc83b            GreenParty   

                                               title           author  \
0            Is Industrial Engineering useful to us?        hiimirony   
1  Never ever turn off your phone: rethinking sec...     soccerskyman   
2     Revolutionary vision and reformist detail work  DependentCarpet   
3            Comedy: billionaires bitchslapping.....          BikkaZz   
4  "We're Green Inside & Outside" ~ Pittsburgh Gr...   faithslayer202   

                                                 url  \
0  https://www.reddit.com/r/LibertarianSocialism/...   
1  https://www.reddit.com/r/Anarchism/comments/8u...   
2  https://www.reddit.com/r/SocialDemocracy/comme...   
3  https://www.reddit.com/r/Capitalism/comments/u...   
4  https://www.reddit.com/r/GreenParty/comments/k...   

             

Here we are using the fetched data to generate a dataset of possible claims for climate skeptics

In [22]:
df_one = pd.read_csv("raw_data/climate_skeptics.csv") #hot 
print(df_one.shape)
df_two = pd.read_csv("raw_data/climate_skeptics_two.csv") #new
print(df_two.shape)
df_three = pd.read_csv("raw_data/climate_skeptics_three.csv") #top
print(df_three.shape)


(1000, 7)
(993, 7)
(1000, 7)


In [59]:
# combine the dataframes
all_df = pd.concat([df_one, df_two, df_three])
print(all_df.shape)

(2993, 7)


In [60]:
#drop the duplicates
all_df = all_df.drop_duplicates(subset=['id'])
print(all_df.shape)

(1939, 7)


In [82]:
# This contains purely text that is good to analyze

#drop NaN from selftext
text_df = all_df.dropna(subset=['selftext'])

#make it into a list and split it by space
text_df['selftext'] = text_df['selftext'].str.split()


print(text_df['selftext'].head())
print(text_df.head())


265    [So,, there's, this, post, on, /r/philosophy, ...
314        [I, hate, those, straws, with, a, passion...]
336    [This, is, something, I've, been, thinking, ab...
358    [The, age, of, independent, truth, seekers, is...
371    [Let, me, start, by, saying, I, believe, in, c...
Name: selftext, dtype: object
                                                 title  \
265           The Lefty's On Reddit are Out of Control   
314                                     Do as I say...   
336  Watch as the climate agenda disappears from media   
358  Man made warming is a hoax, period. But that’s...   
371  I've been driven here due to the cult-like att...   

                                              selftext       id  score  \
265  [So,, there's, this, post, on, /r/philosophy, ...   f1sxv8    226   
314      [I, hate, those, straws, with, a, passion...]  12g6h4b    207   
336  [This, is, something, I've, been, thinking, ab...   fgawu4    205   
358  [The, age, of, independent, truth, 

/Users/liviagimenes/cs1951a_venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [83]:
# this is trying to get titles that might be good to analyze
title_df = all_df.dropna(subset=['title'], axis=0)

#get all of the titles into a list 
title_list = list(title_df['title'].tolist())
print(title_list)

#filter the list to make it a list of list with words
title_list = [title.split() for title in title_list]
print(title_list)

#filter the list to not include list that have less than 15 words
title_list = [title for title in title_list if len(title) > 15]
print(title_list)
print(len(title_list))


['Hypocrisy', 'Its all about the carbon TAX SCAM', 'Yep', "I think I'll wear a fur coat to protest the sale of fur", 'Look', 'hmm, I wonder why the media is only pushing climate change propaganda in the West', 'I wonder if she wants them "up against a wall?"', 'The hockey stick graph is real', 'Fun Fact', 'Climate alarmism couldn’t exist without hypocrisy.', 'You only got 12 years to avoid disaster', "Scientists say meat is crucial for human health and call for the end of pushing 'zealotry' veganism", '“Let’s fight climate change!”', 'Alarmism won´t go extinct', 'Who are we gonna believe?', 'Who Wants To Stop Climate Change?', 'The double standards.', 'Carbon-Hypocrites', 'The alt Left turns a blind eye to CHINA!', 'See the heat. Feel the heat.', 'Greatest Actor of all time with DiCaprio!', 'climate UN changed', "One day, she'll realize it...", 'But Muh Prius (eyes closed)', 'There, that was easy.', 'Greta has no idea how privileged she is', 'I changed my mind, guys!', 'Stolen childhoo

In [86]:
#make a df that has a claim column and if text or title
df_one = pd.DataFrame(columns=['claim', 'type'])

#insert title_list into claim and set all of the type to title
df_one['claim'] = list(title_list)
df_one['type'] = 'title'

#add the text_df[selftext] to the df, and set all of the type to text
df_two = pd.DataFrame(columns=['claim', 'type'])
df_two['claim'] = list(text_df['selftext'].tolist())
df_two['type'] = 'text'

#combine the two df
df = pd.concat([df_one, df_two])
print(df.head())
print(df.shape)

                                               claim   type
0  [Scientists, say, meat, is, crucial, for, huma...  title
1  [Wikipedia, Deletes, "List, of, Scientists, wh...  title
2  [More, failed, points, of, no, return,, Greta'...  title
3  [If, you're, mentally, ill, and, can't, tell, ...  title
4  ["Climate, change, is, real,, it, is, caused, ...  title
(655, 2)


In [87]:
# saving it to a csv
df.to_csv("clean/climate_skeptics_all.csv", index=False)